In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, Rescaling
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import image_dataset_from_directory

from tensorflow_hub import KerasLayer

import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pathlib

# DECLARING VARIABLES

In [14]:
DATA_PATH = pathlib.Path('/content/drive/MyDrive/Colab_Notebooks/projects/mask_detection/data')
IMG_SIZE = (224, 224)
BATCH_SIZE = 20

# LOADING THE IMAGE DATASET

In [15]:
train_ds = image_dataset_from_directory(
    
    DATA_PATH,
    validation_split = 0.2,
    subset = "training",
    seed=123,
    image_size = IMG_SIZE,
    batch_size = BATCH_SIZE
)

Found 5285 files belonging to 2 classes.
Using 4228 files for training.


In [16]:
val_ds = image_dataset_from_directory(
    
    DATA_PATH,
    validation_split = 0.2,
    subset = "validation",
    seed=123,
    image_size = IMG_SIZE,
    batch_size = BATCH_SIZE
)

Found 5285 files belonging to 2 classes.
Using 1057 files for validation.


In [17]:
CLASSES = len(train_ds.class_names)
CLASSES

2

In [18]:
for i,j in train_ds:
  print(i.shape)
  print(j.shape)
  print(j)
  break

(20, 224, 224, 3)
(20,)
tf.Tensor([1 1 0 1 0 1 0 1 1 0 0 0 0 1 0 1 1 1 1 0], shape=(20,), dtype=int32)


In [19]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# TRAINING THE MODEL

In [20]:
pre_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [21]:
model = Sequential([
    Rescaling(1./255),
    KerasLayer(pre_model, input_shape=IMG_SIZE+(3,), trainable=False),
    Dense(CLASSES)  
])

In [22]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [23]:
model.fit(train_ds, epochs=10)

Epoch 1/10
212/212 [==============================] - 763s 4s/step - loss: 0.0672 - accuracy: 0.9745
Epoch 2/10
212/212 [==============================] - 5s 25ms/step - loss: 0.0139 - accuracy: 0.9969
Epoch 3/10
212/212 [==============================] - 5s 25ms/step - loss: 0.0086 - accuracy: 0.9983
Epoch 4/10
212/212 [==============================] - 5s 25ms/step - loss: 0.0059 - accuracy: 0.9991
Epoch 5/10
212/212 [==============================] - 5s 25ms/step - loss: 0.0043 - accuracy: 0.9995
Epoch 6/10
212/212 [==============================] - 5s 25ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 7/10
212/212 [==============================] - 5s 25ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 8/10
212/212 [==============================] - 5s 25ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 9/10
212/212 [==============================] - 5s 25ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 10/10
212/212 [==============================] - 5s 25ms/step - loss: 0.0012 - accura

In [24]:
model.evaluate(val_ds)

53/53 [==============================] - 205s 3s/step - loss: 0.0349 - accuracy: 0.9886


[0.034886233508586884, 0.9886471033096313]

In [25]:
model.save(r'/content/drive/MyDrive/Colab_Notebooks/projects/mask_detection/classifier.h5')

In [27]:
saved =  keras.models.load_model(
    r'/content/drive/MyDrive/Colab_Notebooks/projects/mask_detection/classifier.h5',
    custom_objects={'KerasLayer':KerasLayer}
    )